In [2]:
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix, classification_report, f1_score, recall_score
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('seaborn')

data = pd.read_csv('Data/25day_dataset.csv')


C:\Users\danie\AppData\Local\Temp\ipykernel_24388\1430184262.py:5: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn')


Si etichetta il dataset con 4 classi:

    1) Tutti spenti [0]
    2) Solo lavatrice accesa [1]
    3) Solo lavastoviglie accesa [2]
    4) Solo forno acceso [3]

In [3]:
label = []
for i in range(len(data)):
    if data['wahing_machine'].iloc[i]==0 and data['dishwasher'].iloc[i]==0 and data['oven'].iloc[i]==0:
        label.append(0)
    elif data['wahing_machine'].iloc[i]>0:
        label.append(1)
    elif data['dishwasher'].iloc[i]>0:
        label.append(2)
    elif data['oven'].iloc[i]>0:
        label.append(3)
data['Class'] = label

In [4]:
#Droppa le colonne che non servono piu 
data.drop(['wahing_machine', 'dishwasher', 'oven'], axis=1, inplace=True)

di seguito le istruzioni fronite per effettuare il corretto folding:

Test set fold 01: 
Tutte le misure del giorno 2022-01-01


Test set fold 02: 
Tutte le misure del giorno 2022-01-02

Test set fold 03: 
Tutte le misure del giorno 2022-01-03

…..


Test set fold 09: 
Tutte le misure del giorno 2022-01-10
Tutte le misure del giorno 2022-01-11

Test set fold 10: 
Tutte le misure del giorno 2022-01-12
Tutte le misure del giorno 2022-01-13 (dati disponibili fino alle 03:16)


In [5]:
test_set_list=[]
Test_set1 =data[data['DateTime'].str.startswith('2022-01-01')]
Test_set2 =data[data['DateTime'].str.startswith('2022-01-02')]
Test_set3 =data[data['DateTime'].str.startswith('2022-01-03')]
Test_set4 =data[data['DateTime'].str.startswith('2022-01-04')]
Test_set5 =data[data['DateTime'].str.startswith('2022-01-05')]
Test_set6 =data[data['DateTime'].str.startswith('2022-01-06')]
Test_set7 =data[data['DateTime'].str.startswith('2022-01-07')]
Test_set8 =data[(data['DateTime'].str.startswith('2022-01-08')) | (data['DateTime'].str.startswith('2022-01-09'))]
Test_set9 =data[(data['DateTime'].str.startswith('2022-01-10')) | (data['DateTime'].str.startswith('2022-01-11'))]
Test_set10 =data[(data['DateTime'].str.startswith('2022-01-12')) | (data['DateTime'].str.startswith('2022-01-13'))]

In [6]:
test_set_list= [Test_set1, Test_set2, Test_set3, Test_set4, Test_set5, Test_set6, Test_set7, Test_set8, Test_set9, Test_set10]
Test_set=Test_set1.append([Test_set2,Test_set3,Test_set4,Test_set5,Test_set6,Test_set7,Test_set8,Test_set9,Test_set10])
Training_set= data.drop(Test_set.index, axis=0)

C:\Users\danie\AppData\Local\Temp\ipykernel_24388\1407526483.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Test_set=Test_set1.append([Test_set2,Test_set3,Test_set4,Test_set5,Test_set6,Test_set7,Test_set8,Test_set9,Test_set10])


In [8]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(Training_set.drop(['DateTime', 'Class'], axis=1), Training_set.Class, test_size=0.3, random_state= 8) 
len(X_train)

725760

In [20]:
# validation
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

rfc=RandomForestClassifier(random_state=42)
param_grid = { 
    'n_estimators': [100, 300, 500],
    'min_samples_leaf': [50, 100, 150],
    'criterion' : ['gini', 'entropy', 'log_loss']
}
CV_rfc = GridSearchCV(estimator=rfc, param_grid=param_grid, cv= 5)
CV_rfc.fit(X_val, y_val)
CV_rfc.best_params_

{'criterion': 'entropy', 'min_samples_leaf': 50, 'n_estimators': 100}

In [9]:
#1° addestramento
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=100, criterion='entropy', min_samples_leaf=50)
clf.fit(X_train, y_train)
y_preds= clf.predict(Test_set.drop(['DateTime', 'Class'], axis=1))

print("CM:\n" + str(confusion_matrix(Test_set.Class,y_preds)) + "\n")
print(classification_report(Test_set.Class,y_preds))

CM:
[[1053198    3817     579     373]
 [   2874   32858       0       0]
 [   5148     594    8909       0]
 [   6154     804       1    7891]]

              precision    recall  f1-score   support

           0       0.99      1.00      0.99   1057967
           1       0.86      0.92      0.89     35732
           2       0.94      0.61      0.74     14651
           3       0.95      0.53      0.68     14850

    accuracy                           0.98   1123200
   macro avg       0.94      0.76      0.83   1123200
weighted avg       0.98      0.98      0.98   1123200



In [11]:
#2° addestramento
from sklearn.multiclass import OneVsRestClassifier
from sklearn.naive_bayes import GaussianNB


model =GaussianNB()
ovr_g = OneVsRestClassifier(model)
ovr_g.fit(X_train, y_train)
predicted = ovr_g.predict(Test_set.drop(['DateTime', 'Class'], axis=1))

print("CM:\n" + str(confusion_matrix(Test_set.Class,predicted)) + "\n")
print(classification_report(Test_set.Class,predicted))

CM:
[[1007868   25973   10260   13866]
 [   8745   25517     595     875]
 [   3671     529    8151    2300]
 [   4051    4255     456    6088]]

              precision    recall  f1-score   support

           0       0.98      0.95      0.97   1057967
           1       0.45      0.71      0.55     35732
           2       0.42      0.56      0.48     14651
           3       0.26      0.41      0.32     14850

    accuracy                           0.93   1123200
   macro avg       0.53      0.66      0.58   1123200
weighted avg       0.95      0.93      0.94   1123200



In [13]:
#2° addestramento
from sklearn.multiclass import OneVsOneClassifier
from sklearn.naive_bayes import GaussianNB


model =GaussianNB()
ovo_g = OneVsOneClassifier(model)
ovo_g.fit(X_train, y_train)
predicted = ovo_g.predict(Test_set.drop(['DateTime', 'Class'], axis=1))

print("CM:\n" + str(confusion_matrix(Test_set.Class,predicted)) + "\n")
print(classification_report(Test_set.Class,predicted))

CM:
[[1004991   14300   11758   26918]
 [   7816   22951     423    4542]
 [   3557     546    9357    1191]
 [   4051    1074     201    9524]]

              precision    recall  f1-score   support

           0       0.98      0.95      0.97   1057967
           1       0.59      0.64      0.62     35732
           2       0.43      0.64      0.51     14651
           3       0.23      0.64      0.33     14850

    accuracy                           0.93   1123200
   macro avg       0.56      0.72      0.61   1123200
weighted avg       0.96      0.93      0.94   1123200



In [12]:
#3° addestramento
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC


model = SVC(max_iter=1000)
ovr = OneVsRestClassifier(model)
ovr.fit(X_train, y_train)
predicted = ovr.predict(Test_set.drop(['DateTime', 'Class'], axis=1))

print("CM:\n" + str(confusion_matrix(Test_set.Class,predicted)) + "\n")
print(classification_report(Test_set.Class,predicted))

C:\Users\danie\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\svm\_base.py:301: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
C:\Users\danie\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\svm\_base.py:301: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
C:\Users\danie\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\svm\_base.py:301: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
C:\Users\danie\AppData\Local\Packages\PythonSoftwareFoundatio

CM:
[[1049559     152       0    8256]
 [  27828    6226       0    1678]
 [   5126    9415       0     110]
 [   8294       1       0    6555]]



C:\Users\danie\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\danie\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.96      0.99      0.98   1057967
           1       0.39      0.17      0.24     35732
           2       0.00      0.00      0.00     14651
           3       0.39      0.44      0.42     14850

    accuracy                           0.95   1123200
   macro avg       0.44      0.40      0.41   1123200
weighted avg       0.92      0.95      0.93   1123200



C:\Users\danie\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


ensamble classificatori

In [21]:
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import cross_val_score
X_train, X_t, y_train, y_t = train_test_split(Test_set.drop(['DateTime', 'Class'], axis=1), Test_set.Class, test_size=0.01, random_state= 8) 


eclf = VotingClassifier(estimators=[('ovr', ovr), ('rf', clf), ('gnb', ovo_g)],voting='hard')

eclf.fit(X_t,y_t)
predicted = eclf.predict(Test_set.drop(['DateTime', 'Class'], axis=1))

print("CM:\n" + str(confusion_matrix(Test_set.Class,predicted)) + "\n")
print(classification_report(Test_set.Class,predicted))

'''
for clf, label in zip([ovr, clf, ovo_g, eclf], ['one_VS_Rest', 'Random Forest', 'naive Bayes', 'Ensemble']):
    scores = cross_val_score(clf, X_t, y_t, scoring='accuracy', cv=5)
    print("Accuracy: %0.2f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), label))
'''

C:\Users\danie\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\svm\_base.py:301: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
C:\Users\danie\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\svm\_base.py:301: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
C:\Users\danie\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\svm\_base.py:301: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
C:\Users\danie\AppData\Local\Packages\PythonSoftwareFoundatio

CM:
[[1055862    1401      33     671]
 [   9001   26728       0       3]
 [   5176     547    8928       0]
 [   4949       4      37    9860]]

              precision    recall  f1-score   support

           0       0.98      1.00      0.99   1057967
           1       0.93      0.75      0.83     35732
           2       0.99      0.61      0.76     14651
           3       0.94      0.66      0.78     14850

    accuracy                           0.98   1123200
   macro avg       0.96      0.75      0.84   1123200
weighted avg       0.98      0.98      0.98   1123200



'\nfor clf, label in zip([ovr, clf, ovo_g, eclf], [\'one_VS_Rest\', \'Random Forest\', \'naive Bayes\', \'Ensemble\']):\n    scores = cross_val_score(clf, X_t, y_t, scoring=\'accuracy\', cv=5)\n    print("Accuracy: %0.2f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), label))\n'